# Literature Review

A common task while exploring a new topic is to conduct a literature review. In this example we will explore how a multi-agent team can be configured to conduct a _simple_ literature review.

- **Arxiv Search Agent**: Use the Arxiv API to search for papers related to a given topic, write the search results to a file, and generate a set of relevant questions (e.g., number of papers found, authors, publication date, etc.).
- **Report Agent**: Generate a report based on the information collected by the search and stock analysis agents.


First, let us import the necessary modules. 

In [1]:
from autogen_agentchat.agents import CodingAssistantAgent, ToolUseAssistantAgent
from autogen_agentchat.teams.group_chat import RoundRobinGroupChat
from autogen_core.components.models import OpenAIChatCompletionClient
from autogen_core.components.tools import FunctionTool

## Defining Tools 

Next, we will define the tools that the agents will use to perform their tasks. In this case we will define a simple function `search_arxiv` that will use the `arxiv` library to search for papers related to a given topic.  

Finally, we will wrap the functions into a `FunctionTool` class that will allow us to use it as a tool in the agents. 

Note: You will need to set the appropriate environment variables for tools as needed.

In [2]:

def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:
    import os
    import requests
    from dotenv import load_dotenv
    from bs4 import BeautifulSoup
    import time

    load_dotenv()

    api_key = os.getenv('GOOGLE_API_KEY')
    search_engine_id = os.getenv('GOOGLE_SEARCH_ENGINE_ID')
    
    if not api_key or not search_engine_id:
        raise ValueError("API key or Search Engine ID not found in environment variables")

    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        'key': api_key,
        'cx': search_engine_id,
        'q': query,
        'num': num_results
    }

    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print(response.json())
        raise Exception(f"Error in API request: {response.status_code}")

    results = response.json().get('items', [])
    
    def get_page_content(url):
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            text = soup.get_text(separator=' ', strip=True)
            words = text.split()
            content = ''
            for word in words:
                if len(content) + len(word) + 1 > max_chars:
                    break
                content += ' ' + word
            return content.strip()
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return ""

    enriched_results = []
    for item in results:
        body = get_page_content(item['link'])
        enriched_results.append({
            'title': item['title'],
            'link': item['link'],
            'snippet': item['snippet'],
            'body': body
        })
        time.sleep(1)  # Be respectful to the servers

    return enriched_results

def arxiv_search(query: str, max_results: int = 2) -> list:
    """
    Search Arxiv for papers and return the results including abstracts.
    """
    import arxiv  
    client = arxiv.Client()
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )
    
    results = []
    for paper in client.results(search):
        results.append({
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'published': paper.published.strftime('%Y-%m-%d'),
            'abstract': paper.summary,
            'pdf_url': paper.pdf_url,
        })
    
    # # Write results to a file
    # with open('arxiv_search_results.json', 'w') as f:
    #     json.dump(results, f, indent=2)
    
    return results

In [3]:
google_search_tool = FunctionTool(google_search, description="Search Google for information, returns results with a snippet and body content")
arxiv_search_tool = FunctionTool(arxiv_search, description="Search Arxiv for papers related to a given topic, including abstracts")


## Defining Agents 

Next, we will define the agents that will perform the tasks. 

In [4]:
paper_search_agent = ToolUseAssistantAgent(
    name="Paper_Search_Agent",
    tool_schema=[google_search_tool.schema, arxiv_search_tool.schema],
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="Search both Google AND Arxiv for papers related to a given topic, including abstracts", 
    system_message  = "You are a helpful AI assistant. Solve tasks using your tools.",
)
report_agent = CodingAssistantAgent( 
    name="Report_Agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="Generate a report based on a given topic",  
    system_message  = "You are a helpful assistant. Your task is to synthesize data extracted into a high quality literature review including citations and references. You must write a final report that is formated as a literature review with citations. When you done with generating the report, reply with TERMINATE.",
)

team = RoundRobinGroupChat( 
    participants=[paper_search_agent,report_agent], 
    tools=[google_search_tool,arxiv_search_tool],
)

result = await team.run(task="Write a literature review on no code tools for building multi agent ai systems")
result

--------------------------------------------------------------------------------
user:
Write a literature review on no code tools for building multi agent ai systems
--------------------------------------------------------------------------------
Paper_Search_Agent:
[FunctionCall(id='call_CSxkn4WGSszyPKOluRGnxMOl', arguments='{"query":"no code tools for building multi agent ai systems review","num_results":5}', name='google_search')]
--------------------------------------------------------------------------------
tool_agent:
[FunctionExecutionResult(content='[{\'title\': \'Building GenAI Agents - The approach\', \'link\': \'https://www.linkedin.com/pulse/building-genai-agents-approach-vikram-ekambaram-7el4e\', \'snippet\': \'Jul 1, 2024 ... ... build an agent using a low code / no code tool. I use a ... How do you design incentives and rewards for AI agents in multi-agent systems?\', \'body\': \'Building GenAI Agents - The approach Agree & Join LinkedIn By clicking Continue to join or 

TeamRunResult(messages=[TextMessage(source='user', content='Write a literature review on no code tools for building multi agent ai systems'), TextMessage(source='Paper_Search_Agent', content='In recent years, the rise of no-code tools has significantly transformed the landscape of software development, particularly in the domain of artificial intelligence (AI) and multi-agent systems. These tools empower users with limited programming backgrounds to develop complex applications and AI systems efficiently. This literature review explores various no-code tools and frameworks aimed at building multi-agent AI systems, highlighting their methodologies, functionalities, and impacts on the field.\n\n### 1. Overview of No-Code Tools for Multi-Agent AI Systems\n\nNo-code development platforms provide graphical interfaces and pre-built functionalities that allow users to design, build, and deploy applications without writing code. In the context of multi-agent systems, these tools enable users t